## Atmosphere

### Day 18 - 30 Day Map Challenge

Over Thanksgiving I was tasked with digging out the bunny ears antenna so some of our relatives could watch sportball which got me thinking about data for TV service. I'm a big fan of the [Public Radio Atlas](https://publicradioatlas.org/) which makes beautiful maps showing Public Radio service areas using data from the FCC so I checked to see if similar data is available for broadcast television in the US. And it is! Using the [FCC's TV Service Contours dataset](https://www.fcc.gov/media/television/tv-service-contour-data-points) I put together this map looking at TV station density in the US. 


![](https://raw.githubusercontent.com/johnymontana/30-day-map-challenge/03b921e98df80984331defe812ec832ebeefbed3/img/18-atmosphere-final.png)


I tried a few ways to visualize the contour data directly, but that proved difficult at large scale and I was looking for an excuse to use H3 hexagons anyway.

To follow along create a free account in [Wherobots Cloud.](https://www.wherobots.services/)

In [ ]:
# All other dependencies are installed by default in Wherobots Cloud
!pip install mapclassify

In [ ]:
from sedona.spark import *
import geopandas
import matplotlib.pyplot as plt

In [ ]:
config = SedonaContext.builder().appName('fcc')\
    .config("spark.hadoop.fs.s3a.bucket.wherobots-geodata.aws.credentials.provider","org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")\
    .getOrCreate()
sedona = SedonaContext.create(config)

I downloaded the `TV_service_contour_current.txt` file from the FCC TV Service Contour dataset site and uploaded it to Wherobots Cloud.

![](https://raw.githubusercontent.com/johnymontana/30-day-map-challenge/03b921e98df80984331defe812ec832ebeefbed3/img/atmosphere/files.png)

The file is a pipe delimited flatfile with a few attributes such as the application_id, type of TV service, location of the transmitter site, and then coordinates for the extent of the service contour for each degree from 0-359. We can read this as a CSV file, but then we'll need to figure out how to convert the 360 columns each with a single coordinate into a Polygon geometry to represent the service area of each TV station.

In [ ]:
S3_URL_TV = "s3://wherobots-geodata/TV_service_contour_current.txt"

In [ ]:
tv_df = sedona.read.format('csv').option('header', 'true').option('delimiter', '|').load(S3_URL_TV)

In [ ]:
tv_df.createOrReplaceTempView("tv")

In [ ]:
tv_df.printSchema()

```
root
 |-- application_id: string (nullable = true)
 |-- service: string (nullable = true)
 |-- lms_application_id: string (nullable = true)
 |-- dts_site_number: string (nullable = true)
 |-- transmitter_site: string (nullable = true)
 |-- 0: string (nullable = true)
 |-- 1: string (nullable = true)
 |-- 2: string (nullable = true)
 |-- 3: string (nullable = true)
 |-- 4: string (nullable = true)
 |-- 5: string (nullable = true)
 |-- 6: string (nullable = true)
 |-- 7: string (nullable = true)
 |-- 8: string (nullable = true)
 |-- 9: string (nullable = true)
 |-- 10: string (nullable = true)
 |-- 11: string (nullable = true)
 |-- 12: string (nullable = true)
 |-- 13: string (nullable = true)
 |-- 14: string (nullable = true)
 |-- 15: string (nullable = true)
 |-- 16: string (nullable = true)
 |-- 17: string (nullable = true)
 |-- 18: string (nullable = true)
 |-- 19: string (nullable = true)
 |-- 20: string (nullable = true)
 |-- 21: string (nullable = true)
 |-- 22: string (nullable = true)
 |-- 23: string (nullable = true)
 |-- 24: string (nullable = true)
 |-- 25: string (nullable = true)
 |-- 26: string (nullable = true)
 |-- 27: string (nullable = true)
 |-- 28: string (nullable = true)
 |-- 29: string (nullable = true)
 |-- 30: string (nullable = true)
 |-- 31: string (nullable = true)
 |-- 32: string (nullable = true)
 |-- 33: string (nullable = true)
 |-- 34: string (nullable = true)
 |-- 35: string (nullable = true)
 |-- 36: string (nullable = true)
 |-- 37: string (nullable = true)
 |-- 38: string (nullable = true)
 |-- 39: string (nullable = true)
 |-- 40: string (nullable = true)
 |-- 41: string (nullable = true)
 |-- 42: string (nullable = true)
 |-- 43: string (nullable = true)
 |-- 44: string (nullable = true)
 |-- 45: string (nullable = true)
 |-- 46: string (nullable = true)
 |-- 47: string (nullable = true)
 |-- 48: string (nullable = true)
 |-- 49: string (nullable = true)
 |-- 50: string (nullable = true)
 |-- 51: string (nullable = true)
 |-- 52: string (nullable = true)
 |-- 53: string (nullable = true)
 |-- 54: string (nullable = true)
 |-- 55: string (nullable = true)
 |-- 56: string (nullable = true)
 |-- 57: string (nullable = true)
 |-- 58: string (nullable = true)
 |-- 59: string (nullable = true)
 |-- 60: string (nullable = true)
 |-- 61: string (nullable = true)
 |-- 62: string (nullable = true)
 |-- 63: string (nullable = true)
 |-- 64: string (nullable = true)
 |-- 65: string (nullable = true)
 |-- 66: string (nullable = true)
 |-- 67: string (nullable = true)
 |-- 68: string (nullable = true)
 |-- 69: string (nullable = true)
 |-- 70: string (nullable = true)
 |-- 71: string (nullable = true)
 |-- 72: string (nullable = true)
 |-- 73: string (nullable = true)
 |-- 74: string (nullable = true)
 |-- 75: string (nullable = true)
 |-- 76: string (nullable = true)
 |-- 77: string (nullable = true)
 |-- 78: string (nullable = true)
 |-- 79: string (nullable = true)
 |-- 80: string (nullable = true)
 |-- 81: string (nullable = true)
 |-- 82: string (nullable = true)
 |-- 83: string (nullable = true)
 |-- 84: string (nullable = true)
 |-- 85: string (nullable = true)
 |-- 86: string (nullable = true)
 |-- 87: string (nullable = true)
 |-- 88: string (nullable = true)
 |-- 89: string (nullable = true)
 |-- 90: string (nullable = true)
 |-- 91: string (nullable = true)
 |-- 92: string (nullable = true)
 |-- 93: string (nullable = true)
 |-- 94: string (nullable = true)
 |-- 95: string (nullable = true)
 |-- 96: string (nullable = true)
 |-- 97: string (nullable = true)
 |-- 98: string (nullable = true)
 |-- 99: string (nullable = true)
 |-- 100: string (nullable = true)
 |-- 101: string (nullable = true)
 |-- 102: string (nullable = true)
 |-- 103: string (nullable = true)
 |-- 104: string (nullable = true)
 |-- 105: string (nullable = true)
 |-- 106: string (nullable = true)
 |-- 107: string (nullable = true)
 |-- 108: string (nullable = true)
 |-- 109: string (nullable = true)
 |-- 110: string (nullable = true)
 |-- 111: string (nullable = true)
 |-- 112: string (nullable = true)
 |-- 113: string (nullable = true)
 |-- 114: string (nullable = true)
 |-- 115: string (nullable = true)
 |-- 116: string (nullable = true)
 |-- 117: string (nullable = true)
 |-- 118: string (nullable = true)
 |-- 119: string (nullable = true)
 |-- 120: string (nullable = true)
 |-- 121: string (nullable = true)
 |-- 122: string (nullable = true)
 |-- 123: string (nullable = true)
 |-- 124: string (nullable = true)
 |-- 125: string (nullable = true)
 |-- 126: string (nullable = true)
 |-- 127: string (nullable = true)
 |-- 128: string (nullable = true)
 |-- 129: string (nullable = true)
 |-- 130: string (nullable = true)
 |-- 131: string (nullable = true)
 |-- 132: string (nullable = true)
 |-- 133: string (nullable = true)
 |-- 134: string (nullable = true)
 |-- 135: string (nullable = true)
 |-- 136: string (nullable = true)
 |-- 137: string (nullable = true)
 |-- 138: string (nullable = true)
 |-- 139: string (nullable = true)
 |-- 140: string (nullable = true)
 |-- 141: string (nullable = true)
 |-- 142: string (nullable = true)
 |-- 143: string (nullable = true)
 |-- 144: string (nullable = true)
 |-- 145: string (nullable = true)
 |-- 146: string (nullable = true)
 |-- 147: string (nullable = true)
 |-- 148: string (nullable = true)
 |-- 149: string (nullable = true)
 |-- 150: string (nullable = true)
 |-- 151: string (nullable = true)
 |-- 152: string (nullable = true)
 |-- 153: string (nullable = true)
 |-- 154: string (nullable = true)
 |-- 155: string (nullable = true)
 |-- 156: string (nullable = true)
 |-- 157: string (nullable = true)
 |-- 158: string (nullable = true)
 |-- 159: string (nullable = true)
 |-- 160: string (nullable = true)
 |-- 161: string (nullable = true)
 |-- 162: string (nullable = true)
 |-- 163: string (nullable = true)
 |-- 164: string (nullable = true)
 |-- 165: string (nullable = true)
 |-- 166: string (nullable = true)
 |-- 167: string (nullable = true)
 |-- 168: string (nullable = true)
 |-- 169: string (nullable = true)
 |-- 170: string (nullable = true)
 |-- 171: string (nullable = true)
 |-- 172: string (nullable = true)
 |-- 173: string (nullable = true)
 |-- 174: string (nullable = true)
 |-- 175: string (nullable = true)
 |-- 176: string (nullable = true)
 |-- 177: string (nullable = true)
 |-- 178: string (nullable = true)
 |-- 179: string (nullable = true)
 |-- 180: string (nullable = true)
 |-- 181: string (nullable = true)
 |-- 182: string (nullable = true)
 |-- 183: string (nullable = true)
 |-- 184: string (nullable = true)
 |-- 185: string (nullable = true)
 |-- 186: string (nullable = true)
 |-- 187: string (nullable = true)
 |-- 188: string (nullable = true)
 |-- 189: string (nullable = true)
 |-- 190: string (nullable = true)
 |-- 191: string (nullable = true)
 |-- 192: string (nullable = true)
 |-- 193: string (nullable = true)
 |-- 194: string (nullable = true)
 |-- 195: string (nullable = true)
 |-- 196: string (nullable = true)
 |-- 197: string (nullable = true)
 |-- 198: string (nullable = true)
 |-- 199: string (nullable = true)
 |-- 200: string (nullable = true)
 |-- 201: string (nullable = true)
 |-- 202: string (nullable = true)
 |-- 203: string (nullable = true)
 |-- 204: string (nullable = true)
 |-- 205: string (nullable = true)
 |-- 206: string (nullable = true)
 |-- 207: string (nullable = true)
 |-- 208: string (nullable = true)
 |-- 209: string (nullable = true)
 |-- 210: string (nullable = true)
 |-- 211: string (nullable = true)
 |-- 212: string (nullable = true)
 |-- 213: string (nullable = true)
 |-- 214: string (nullable = true)
 |-- 215: string (nullable = true)
 |-- 216: string (nullable = true)
 |-- 217: string (nullable = true)
 |-- 218: string (nullable = true)
 |-- 219: string (nullable = true)
 |-- 220: string (nullable = true)
 |-- 221: string (nullable = true)
 |-- 222: string (nullable = true)
 |-- 223: string (nullable = true)
 |-- 224: string (nullable = true)
 |-- 225: string (nullable = true)
 |-- 226: string (nullable = true)
 |-- 227: string (nullable = true)
 |-- 228: string (nullable = true)
 |-- 229: string (nullable = true)
 |-- 230: string (nullable = true)
 |-- 231: string (nullable = true)
 |-- 232: string (nullable = true)
 |-- 233: string (nullable = true)
 |-- 234: string (nullable = true)
 |-- 235: string (nullable = true)
 |-- 236: string (nullable = true)
 |-- 237: string (nullable = true)
 |-- 238: string (nullable = true)
 |-- 239: string (nullable = true)
 |-- 240: string (nullable = true)
 |-- 241: string (nullable = true)
 |-- 242: string (nullable = true)
 |-- 243: string (nullable = true)
 |-- 244: string (nullable = true)
 |-- 245: string (nullable = true)
 |-- 246: string (nullable = true)
 |-- 247: string (nullable = true)
 |-- 248: string (nullable = true)
 |-- 249: string (nullable = true)
 |-- 250: string (nullable = true)
 |-- 251: string (nullable = true)
 |-- 252: string (nullable = true)
 |-- 253: string (nullable = true)
 |-- 254: string (nullable = true)
 |-- 255: string (nullable = true)
 |-- 256: string (nullable = true)
 |-- 257: string (nullable = true)
 |-- 258: string (nullable = true)
 |-- 259: string (nullable = true)
 |-- 260: string (nullable = true)
 |-- 261: string (nullable = true)
 |-- 262: string (nullable = true)
 |-- 263: string (nullable = true)
 |-- 264: string (nullable = true)
 |-- 265: string (nullable = true)
 |-- 266: string (nullable = true)
 |-- 267: string (nullable = true)
 |-- 268: string (nullable = true)
 |-- 269: string (nullable = true)
 |-- 270: string (nullable = true)
 |-- 271: string (nullable = true)
 |-- 272: string (nullable = true)
 |-- 273: string (nullable = true)
 |-- 274: string (nullable = true)
 |-- 275: string (nullable = true)
 |-- 276: string (nullable = true)
 |-- 277: string (nullable = true)
 |-- 278: string (nullable = true)
 |-- 279: string (nullable = true)
 |-- 280: string (nullable = true)
 |-- 281: string (nullable = true)
 |-- 282: string (nullable = true)
 |-- 283: string (nullable = true)
 |-- 284: string (nullable = true)
 |-- 285: string (nullable = true)
 |-- 286: string (nullable = true)
 |-- 287: string (nullable = true)
 |-- 288: string (nullable = true)
 |-- 289: string (nullable = true)
 |-- 290: string (nullable = true)
 |-- 291: string (nullable = true)
 |-- 292: string (nullable = true)
 |-- 293: string (nullable = true)
 |-- 294: string (nullable = true)
 |-- 295: string (nullable = true)
 |-- 296: string (nullable = true)
 |-- 297: string (nullable = true)
 |-- 298: string (nullable = true)
 |-- 299: string (nullable = true)
 |-- 300: string (nullable = true)
 |-- 301: string (nullable = true)
 |-- 302: string (nullable = true)
 |-- 303: string (nullable = true)
 |-- 304: string (nullable = true)
 |-- 305: string (nullable = true)
 |-- 306: string (nullable = true)
 |-- 307: string (nullable = true)
 |-- 308: string (nullable = true)
 |-- 309: string (nullable = true)
 |-- 310: string (nullable = true)
 |-- 311: string (nullable = true)
 |-- 312: string (nullable = true)
 |-- 313: string (nullable = true)
 |-- 314: string (nullable = true)
 |-- 315: string (nullable = true)
 |-- 316: string (nullable = true)
 |-- 317: string (nullable = true)
 |-- 318: string (nullable = true)
 |-- 319: string (nullable = true)
 |-- 320: string (nullable = true)
 |-- 321: string (nullable = true)
 |-- 322: string (nullable = true)
 |-- 323: string (nullable = true)
 |-- 324: string (nullable = true)
 |-- 325: string (nullable = true)
 |-- 326: string (nullable = true)
 |-- 327: string (nullable = true)
 |-- 328: string (nullable = true)
 |-- 329: string (nullable = true)
 |-- 330: string (nullable = true)
 |-- 331: string (nullable = true)
 |-- 332: string (nullable = true)
 |-- 333: string (nullable = true)
 |-- 334: string (nullable = true)
 |-- 335: string (nullable = true)
 |-- 336: string (nullable = true)
 |-- 337: string (nullable = true)
 |-- 338: string (nullable = true)
 |-- 339: string (nullable = true)
 |-- 340: string (nullable = true)
 |-- 341: string (nullable = true)
 |-- 342: string (nullable = true)
 |-- 343: string (nullable = true)
 |-- 344: string (nullable = true)
 |-- 345: string (nullable = true)
 |-- 346: string (nullable = true)
 |-- 347: string (nullable = true)
 |-- 348: string (nullable = true)
 |-- 349: string (nullable = true)
 |-- 350: string (nullable = true)
 |-- 351: string (nullable = true)
 |-- 352: string (nullable = true)
 |-- 353: string (nullable = true)
 |-- 354: string (nullable = true)
 |-- 355: string (nullable = true)
 |-- 356: string (nullable = true)
 |-- 357: string (nullable = true)
 |-- 358: string (nullable = true)
 |-- 359: string (nullable = true)
 |-- ^: string (nullable = true)
 |-- _c366: string (nullable = true)
```

Now to create a Polygon geometry from the 360 coordinates, each in a separate column. We could do this using the DataFrame API, but I was searching for a pure SQL solution. This is perhaps not the most elegant solution but I chose to programatically construct a SQL fragment that would parse each coordinate value using SQL's `SPLIT` string function and then insert that SQL fragment into a larger SQL statement.

In [ ]:
cols = ','.join([str(f"SPLIT(`{x}`, ',')[1], ' ', SPLIT(`{x}`, ',')[0]" + (",' ,'" if int(x) < 359 else '')) for x in tv_df.schema.names[5:-2]])

In [ ]:
tv_df = sedona.sql(f"SELECT application_id, service, lms_application_id, dts_site_number, ST_GeomFromWKT(CONCAT('POLYGON ((' , {cols}, '))')) AS geometry  FROM tv")

In [ ]:
tv_df.show()

```
+--------------+-------+--------------------+---------------+--------------------+
|application_id|service|  lms_application_id|dts_site_number|            geometry|
+--------------+-------+--------------------+---------------+--------------------+
|    2036528   |    LPD|25076ff3729b1e0a0...|             01|POLYGON ((-154.84...|
|    2020763   |    DTV|25076f9169e0309d0...|             01|POLYGON ((-87.135...|
|    2051473   |    LPD|25076ff386f28a460...|             01|POLYGON ((-114.82...|
|    2047477   |    LPD|25076ff380754a570...|             01|POLYGON ((-114.82...|
|    2047332   |    LPD|25076ff38197b72e0...|             01|POLYGON ((-120.2 ...|
|    2045786   |    LPD|25076ff3818b1da20...|             01|POLYGON ((-116.22...|
|    2042394   |    LPD|25076ff379edbd630...|             01|POLYGON ((-116.23...|
|    2047209   |    LPD|25076f9181644b630...|             01|POLYGON ((-118.07...|
|    1636431   |    LPT|5c099f4617394bd0a...|             01|POLYGON ((-118.07...|
|    2050793   |    LPD|25076f91858430210...|             01|POLYGON ((-122.63...|
|    2049140   |    LPD|25076f9181f47ea90...|             01|POLYGON ((-122.63...|
|    2001458   |    LPD|25076f914dfdd8ab0...|             01|POLYGON ((-121.83...|
|    1505135   |    LPT|8454aac897db42e39...|             01|POLYGON ((-124.04...|
|    1262099   |    LPT|3147136286024a88b...|             01|POLYGON ((-122.98...|
|    2036969   |    LPD|25076f91732a46140...|             01|POLYGON ((-105.23...|
|    1422197   |    LPT|d9f7f05a979a4b058...|             01|POLYGON ((-108.52...|
|    1424848   |    LPT|f8cb3756787b40d7a...|             01|POLYGON ((-107.25...|
|    2002433   |    DTV|25076ff3524603200...|             01|POLYGON ((-108.56...|
|    1439031   |    LPT|96c956b5f07741a08...|             01|POLYGON ((-108.02...|
|    1439043   |    LPT|957fab47770747899...|             01|POLYGON ((-108.68...|
+--------------+-------+--------------------+---------------+--------------------+
only showing top 20 rows
```

In [ ]:
tv_df.createOrReplaceTempView("tv")

Now we have a Spatial DataFrame where each row represents a TV station and the `geometry` column is a polygon that represents the service area of that TV station. 

We can visualize the contours individually using `SedonaKepler`

In [ ]:
SedonaKepler.create_map(tv_df, "TV Service Contours")

![](https://raw.githubusercontent.com/johnymontana/30-day-map-challenge/03b921e98df80984331defe812ec832ebeefbed3/img/atmosphere/kepler-contours.png)

Instead of visualizing each contour, let's see if we can visualize the density of overlapping TV stations using H3 hexagons. We can use the H3 hexagon functions in SedonaDB to create a hexagon tesselation over our entire service area, then use a SQL `GROUP BY` to count the number of TV station geometries intersecting with each hexagon.

In [ ]:
h3_df = sedona.sql("""
SELECT COUNT(*) AS num, ST_H3ToGeom(ST_H3CellIDs(geometry, 2, false)) as geometry, ST_H3CellIDs(geometry, 2, false) as h3 
FROM tv 
GROUP BY h3
""")

In [ ]:
h3_df.show(5)

```
+---+--------------------+--------------------+
|num|            geometry|                  h3|
+---+--------------------+--------------------+
| 11|MULTIPOLYGON (((-...|[585703247046508543]|
| 31|MULTIPOLYGON (((-...|[586138653651107839]|
| 17|MULTIPOLYGON (((-...|[586236510185979903]|
| 17|MULTIPOLYGON (((-...|[586166141441802239]|
| 75|MULTIPOLYGON (((-...|[586164492174360575]|
+---+--------------------+--------------------+
only showing top 5 rows
```

Our new DataFrame shows the number of TV stations in each hexagon.

In [ ]:
SedonaKepler.create_map(h3_df)

![](https://raw.githubusercontent.com/johnymontana/30-day-map-challenge/03b921e98df80984331defe812ec832ebeefbed3/img/atmosphere/kepler-hexagons.png)

We can create an interactive choropleth to visualize the data using `SedonaPyDeck`.

In [ ]:
SedonaPyDeck.create_choropleth_map(h3_df, plot_col="num")

![](https://raw.githubusercontent.com/johnymontana/30-day-map-challenge/03b921e98df80984331defe812ec832ebeefbed3/img/atmosphere/pydeck.png)

Let's also create a choropleth using matplotlib.

In [ ]:
h3_df = h3_df.filter("ST_XMin(geometry) < 0")
h3_gdf = geopandas.GeoDataFrame(h3_df.toPandas(), geometry='geometry')

In [ ]:
h3_gdf.plot()

![](https://raw.githubusercontent.com/johnymontana/30-day-map-challenge/03b921e98df80984331defe812ec832ebeefbed3/img/atmosphere/matplotlib.png)

Note the distortion of the hexagons especially at larger latitudes. Let's use an equal area projection to preserve the area of each hexagon (we'll lose a bit of the shape at the extremes, but area will be preserved which is typically what we want when creating a choropleth map)

In [ ]:
h3_gdf.crs = {"init": "epsg:4326"}
h3_gdf.to_crs(epsg=9822)

In [ ]:
ax = h3_gdf.plot(
    column="num",
    scheme="JenksCaspall",
    cmap="YlGnBu",
    legend=True,
    legend_kwds={"title": "Number of TV stations", "fontsize": 20, "title_fontsize": 20},
    figsize=(24,18)
)

ax.set_axis_off()
ax.set_title("US TV Station Density", fontsize=40)
ax.annotate("Data from FCC TV Service Countours Dataset", (-175,15))

plt.savefig("fcc.png", dpi=300)

![](https://raw.githubusercontent.com/johnymontana/30-day-map-challenge/03b921e98df80984331defe812ec832ebeefbed3/img/18-atmosphere-final.png)